In [1]:
%load_ext autoreload
%autoreload 2
import geopandas as gpd
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import matplotlib.pyplot as plt
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import itertools
from shapely.geometry import Point

import glob

In [20]:
shapefile = "/home/mlopez/EXEC/Shapefiles/Secteurs_Operations_Regionales.shp"

# Accessing shapefiles

In [2]:
shfolder = "/home/mlopez/EXEC/Shapefiles/"
out_folder = "/home/mlopez/EXEC/Grids-polygons-regions/"

In [12]:
shapefiles =  glob.glob(shfolder+"*.shp")
shapefiles

['/home/mlopez/EXEC/Shapefiles/DDE_STF_20K_REG_FOR_VUE_S.shp',
 '/home/mlopez/EXEC/Shapefiles/DDE_STF_20K_UA_PER_VUE_S.shp',
 '/home/mlopez/EXEC/Shapefiles/DOM_BIO.shp',
 '/home/mlopez/EXEC/Shapefiles/REG_ECO.shp',
 '/home/mlopez/EXEC/Shapefiles/SDOM_BIO.shp',
 '/home/mlopez/EXEC/Shapefiles/Secteurs_Operations_Regionales.shp',
 '/home/mlopez/EXEC/Shapefiles/SREG_ECO.shp',
 '/home/mlopez/EXEC/Shapefiles/territoire_guide.shp']

In [32]:
shnames1 = [str(sub.split('Shapefiles/')[1]) for sub in shapefiles] 
shnames1

['DDE_STF_20K_REG_FOR_VUE_S.shp',
 'DDE_STF_20K_UA_PER_VUE_S.shp',
 'DOM_BIO.shp',
 'REG_ECO.shp',
 'SDOM_BIO.shp',
 'Secteurs_Operations_Regionales.shp',
 'SREG_ECO.shp',
 'territoire_guide.shp']

In [34]:
shnames = [str(sub.split('.shp')[0]) for sub in shnames1] 
shnames

['DDE_STF_20K_REG_FOR_VUE_S',
 'DDE_STF_20K_UA_PER_VUE_S',
 'DOM_BIO',
 'REG_ECO',
 'SDOM_BIO',
 'Secteurs_Operations_Regionales',
 'SREG_ECO',
 'territoire_guide']

In [35]:
shnames[1]

'DDE_STF_20K_UA_PER_VUE_S'

# Accessing data files

In [8]:
datafolder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"

variable = "*rcp85_tg_mean_annual.nc"


# Convert all lat lon of 11 models into points 

In [16]:
# It takes 20s
%time dfpoints = pcdf.data_to_points(datafolder, variable)

CPU times: user 1.99 s, sys: 214 ms, total: 2.2 s
Wall time: 2.39 s


In [17]:
dfpoints

,lat,lon,geometry
0,66.623306,-89.045212,POINT (-89.04521 66.62331)
1,66.623306,-88.961884,POINT (-88.96188 66.62331)
2,66.623306,-88.878548,POINT (-88.87855 66.62331)
3,66.623306,-88.795219,POINT (-88.79522 66.62331)
4,66.623306,-88.711891,POINT (-88.71189 66.62331)
...,...,...,...
133115,40.041039,-54.796581,POINT (-54.79658 40.04104)
133116,40.041039,-54.713249,POINT (-54.71325 40.04104)
133117,40.041039,-54.629921,POINT (-54.62992 40.04104)
133118,40.041039,-54.546589,POINT (-54.54659 40.04104)


# Create polygons

In [18]:
#takes 8s
%time dfpoints.geometry  = pcdf.process_points_poly(dfpoints)

CPU times: user 1.48 s, sys: 90.9 ms, total: 1.57 s
Wall time: 1.56 s


In [19]:
dfpoints.head()

,lat,lon,geometry
0,66.623306,-89.045212,"POLYGON ((-89.08688 66.58164, -89.00355 66.581..."
1,66.623306,-88.961884,"POLYGON ((-89.00355 66.58164, -88.92022 66.581..."
2,66.623306,-88.878548,"POLYGON ((-88.92021 66.58164, -88.83688 66.581..."
3,66.623306,-88.795219,"POLYGON ((-88.83688 66.58164, -88.75356 66.581..."
4,66.623306,-88.711891,"POLYGON ((-88.75356 66.58164, -88.67023 66.581..."


In [21]:
### TAKES 10 min for all the regions TG
%time  dfpolyshape = pcdf.make_grid(dfpoints, shapefile) #10 min with overlay #1miin with sjoin

CPU times: user 14min, sys: 1.48 s, total: 14min 2s
Wall time: 13min 44s


In [25]:
dfpolyshape.head()

,lat,lon,OBJECTID,NO_REG_FOR,NM_REG_FOR,NO_UG_RAF,SUP_KM2,DATE_PUBLI,Shape_Leng,Shape_Area,D_GENERAL,geometry
0,53.040520,-66.462784,8,09,COTE-NORD,"093, 094, 097",379435.8,2019-11-30,5.360494e+06,3.744732e+11,Nord-est,"MULTIPOLYGON (((-66.42112 53.03552, -66.42112 ..."
1,53.040520,-66.379448,8,09,COTE-NORD,"093, 094, 097",379435.8,2019-11-30,5.360494e+06,3.744732e+11,Nord-est,"POLYGON ((-66.42111 52.99885, -66.42111 53.035..."
2,52.957191,-66.629440,8,09,COTE-NORD,"093, 094, 097",379435.8,2019-11-30,5.360494e+06,3.744732e+11,Nord-est,"POLYGON ((-66.58778 52.94277, -66.58778 52.915..."
3,52.957191,-66.546112,8,09,COTE-NORD,"093, 094, 097",379435.8,2019-11-30,5.360494e+06,3.744732e+11,Nord-est,"POLYGON ((-66.58778 52.91553, -66.58778 52.942..."
4,52.957191,-66.462784,8,09,COTE-NORD,"093, 094, 097",379435.8,2019-11-30,5.360494e+06,3.744732e+11,Nord-est,"POLYGON ((-66.50445 52.91553, -66.50445 52.948..."


In [36]:
to_geofeather(dfpolyshape, 'Grid'+shnames[1]+'.feather')

In [ ]:
shnames[1]

In [46]:
i = 3
for shape in shapefiles:
    print (shapefiles[i])
    print(i)
    dfpolyshape = pcdf.make_grid(dfpoints, shapefiles[i])
    print(shape)
    to_geofeather(dfpolyshape, 'Grid'+shnames[i]+'.feather')
    print("geofather done")
    i = i+1

/home/mlopez/EXEC/Shapefiles/REG_ECO.shp
3
/home/mlopez/EXEC/Shapefiles/DDE_STF_20K_REG_FOR_VUE_S.shp
geofather done
/home/mlopez/EXEC/Shapefiles/SDOM_BIO.shp
4


KeyboardInterrupt: 